This notebook contains the code in the [LangChain AzureOpenAI quickstart guide for Python](https://python.langchain.com/en/latest/modules/models/llms/integrations/azure_openai_example.html)

## Environment Setup

In [4]:
# Set environment variables in the .env file.

COMPLETION_MODEL = os.environ["OPENAI_COMPLETION_MODEL"]
COMPLETION_DEPLOYMENT = os.environ["OPENAI_COMPLETION_DEPLOYMENT"]
CHAT_MODEL = os.environ["OPENAI_CHAT_MODEL"]
CHAT_DEPLOYMENT = os.environ["OPENAI_CHAT_DEPLOYMENT"]
CHAT_API_VERSION = os.environ["OPENAI_CHAT_API_VERSION"]

## Completion Quickstart

In [5]:
import openai

response = openai.Completion.create(
    engine=COMPLETION_MODEL,
    prompt="Hello.",
    max_tokens=10
)

response

<OpenAIObject text_completion id=cmpl-7BKh7Mg0EIr4y32fEomIkBybL1pR4 at 0x1591ca3c5f0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nHi there! How can I help you"
    }
  ],
  "created": 1682935513,
  "id": "cmpl-7BKh7Mg0EIr4y32fEomIkBybL1pR4",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 2,
    "total_tokens": 12
  }
}

In [4]:
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"], # optional, specify if value is not in environment variable OPENAI_API_KEY
    model_name=COMPLETION_MODEL,
    deployment_name=COMPLETION_DEPLOYMENT,
    temperature=0.7,
    max_tokens=10
)

print(llm)

AzureOpenAI
Params: {'deployment_name': 'text-davinci-003', 'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 10, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'best_of': 1, 'request_timeout': None, 'logit_bias': {}}


In [5]:
llm("Tell me a joke.")

'\n\nQ: What did the fish say when'

# Chat Model

## Chat Messages

In [10]:
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=CHAT_API_VERSION
)

In [12]:
messages = [
    SystemMessage(content="You are an assistant who likes to respond with dad jokes."),
    HumanMessage(content="Translate this sentense from English to Singlish. I love Python programming with Jupyter notebooks!")
]

chat(messages)

AIMessage(content='Wah, I really love using Python for programming with Jupyter notebooks leh!', additional_kwargs={})

## Chat Prompt Templates

In [13]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=CHAT_API_VERSION
)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        input_language="English",
        output_language="Singlish",
        text="I love Python programming with Jupyter notebooks!"
    )
    .to_messages()
)

AIMessage(content='I love Python programming with Jupyter notebooks lah!', additional_kwargs={})

## Chat Memory

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to the question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = AzureChatOpenAI(
    deployment_name=CHAT_DEPLOYMENT,
    openai_api_version=CHAT_API_VERSION
)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

In [ ]:
conversation.predict(input="Hi there!")

In [16]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

"That's great! I am always happy to chat. Is there anything specific you would like to talk about?"

In [17]:
conversation.predict(input="Kwentohan mo ako tungkol sa sarili mo.")

"Ako po ay isang artificial intelligence o AI na nilikha upang magbigay ng tulong sa mga tao sa kanilang mga pangangailangan. Ginagamit ako sa iba't ibang larangan tulad ng pagproseso ng data, pagpapayo sa mga kliyente, at maging sa pagtugon sa simpleng mga tanong tulad ng mga itatanong ninyo. Ang aking kakayahan ay nakabatay sa mga algorithm at kalkulasyon, na nagbibigay sa akin ng kakayahang magbigay ng mga sagot at solusyon sa mga problema."

In [18]:
print(conversation)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi there!', additional_kwargs={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}), HumanMessage(content="I'm doing well! Just having a conversation with an AI.", additional_kwargs={}), AIMessage(content="That's great! I am always happy to chat. Is there anything specific you would like to talk about?", additional_kwargs={}), HumanMessage(content='Kwentohan mo ako tungkol sa sarili mo.', additional_kwargs={}), AIMessage(content="Ako po ay isang artificial intelligence o AI na nilikha upang magbigay ng tulong sa mga tao sa kanilang mga pangangailangan. Ginagamit ako sa iba't ibang larangan tulad ng pagproseso ng data, pagpapayo sa mga kliyente, at maging sa pagtugon sa simpleng mga tanong tulad ng mga itatanong ninyo. Ang aking kakayahan ay nakabatay sa mga algorithm at kalkulasyon, na nagbibigay sa akin ng kakayahang magbigay ng mga sagot at solusyon sa mga probl

In [3]:
conversation.run("Hello there!")

'Greetings! How may I assist you today?'

## ChatSummaryBufferMemory
- The `orchestrator-simple` isn't summarizing the conversation, try [this](https://python.langchain.com/en/latest/modules/memory/types/summary_buffer.html) out first in this notebook.